In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [69]:
# 2910 reconfinement
nn=1
JE,JI=1.60,1.2
a,b=0.0,0.0#1.5,0.25#0.3,0.2#
# a=-JI*(JE/b)#+0.8
# a=-JI/(JE/b)#-0.3

'''
J=np.zeros((nn*2,nn*2))
J[:,:nn],J[:,nn:]=JE/nn,-JI/nn
J[:nn,:nn],J[nn:,:nn]=J[:nn,:nn]-a/nn,J[nn:,:nn]+a/nn
J[:nn,nn:],J[nn:,nn:]=J[:nn,nn:]+b/nn,J[nn:,nn:]-b/nn
print('2by2 matrix:',J)
lvec,sv,rvech=la.svd(J)
print('sum:',np.sum(sv),'diff:',sv[0]-sv[1])
s1=np.sqrt(2*((JE+b)**2+(JI-a)**2))
s2=np.sqrt(2*((JE-b)**2+(JI+a)**2))
print('sum1:',s1,'; sum2:',s2)
'''

# generate perturbed matrix
nn=600
# # JE,JI=0.8,0.6
# # a,b=0.80,0.2#1.5,-0.3
# a1=-JE*b/JI
# a2=-JE*JI/b
# print('a1-',a1,' a2-',a2)
Jr=np.zeros((nn*2,nn*2))
Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
# Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]-a/nn,Jr[nn:,:nn]+a/nn
# Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
lvec,svr,rvech=la.svd(Jr)
print(' random singular value:',svr[:2])

# add random matrix
x = np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
x_= x+Jr
lvec_,sv_,rvech_=la.svd(x_)
theo_sv=1/svr[:2]+svr[:2]
# print('theoretical solution:',theo_sv,'; numerical solution:',sv_[:2])

rholimit=1/svr[0]+svr[0]
derivphi=1/(1-svr[0]**2)
overlap=-1/derivphi/svr[0]**2
# print('am theo:',1/sv[0]**2)

print('overlap num:',np.sum(lvec_[:,0]*lvec[:,0])**2)




# Tomorrow, check whether the overlap of the perturbed matrix is equal to the original * alpha square.

 random singular value: [2.82842712e+00 2.51212613e-14]
overlap num: 0.8761283878410749


In [74]:
## calculate whether there is an additive relationship
overlap_num=1.0-1/svr[0]**2
alpha = np.sqrt(overlap_num)
rvec_,rvec=rvech_.T,rvech.T
lvar,rvar=np.zeros_like(lvec_[:,0]),np.zeros_like(rvec_[:,0])
lvar[:nn]=lvec_[:nn,0]-alpha*lvec[:nn,0]
lvar[nn:]=lvec_[nn:,0]-alpha*lvec[nn:,0]
print('overlap:',alpha**2)
print('sigma:',rholimit,'; numsigmas:',sv_[0])
print('1/sigma:',1/sv_[0]**2)
plt.figure()
n, bins, patches = plt.hist(lvar[:], 100, density=True, facecolor='g', alpha=0.75)


rvar[:nn]=rvec_[:nn,0]-alpha*rvec[:nn,0]
rvar[nn:]=rvec_[nn:,0]-alpha*rvec[nn:,0]
plt.figure()
n, bins, patches = plt.hist(rvar[:], 100, density=True, facecolor='g', alpha=0.75)
plt.figure()
n, bins, patches = plt.hist(rvec_[:,0], 100, density=True, facecolor='g', alpha=0.75)
print(np.mean(np.sum(lvar*rvar)))
# plt.figure()
# n, bins, patches = plt.hist(lvar[:nn], 100, density=True, facecolor='g', alpha=0.75)
print(np.mean(np.sum(lvar[:nn]*rvar[:nn])))
# plt.figure()
# n, bins, patches = plt.hist(lvar[nn:], 100, density=True, facecolor='g', alpha=0.75)
print(np.mean(np.sum(lvar[nn:]*rvar[nn:])))
print('rvector:',rvec[:2,0],'; ',rvec[nn:nn+2,0])
print('tilte rvector:',rvec_[:10,0],'; ',rvec_[nn:nn+10,0])
#tilte overlap
tilteuv=np.sum(rvec_[:,0]*lvec_[:,0])
orguv=alpha**2*np.sum(rvec[:,0]*lvec[:,0])
cosuv=(1/svr[0]**2)*np.sum(rvec[:,0]*lvec[:,0])

overlap: 0.8749999999999996
sigma: 3.1819805153394594 ; numsigmas: 3.202035340065009
1/sigma: 0.09753214112167677
-0.0021557554204781606
0.0008231584202601689
-0.0029789138407383293
tilte rvector: [-0.03265986 -0.03265986] ;  [0.0244949 0.0244949]
tilte rvector: [-0.02735731 -0.03549696 -0.04522799 -0.03124508 -0.03046251 -0.03884344
 -0.01234629 -0.02786855 -0.05734784 -0.01903706] ;  [0.00585008 0.0067866  0.03829516 0.02833079 0.01775737 0.015754
 0.01489371 0.01419926 0.00404347 0.02658142]


** What's the difference between substract tilde and original **

In [204]:
# 2910 reconfinement
nn=1
JE,JI=0.60,0.4
a,b=0.0,0.0#1.5,0.25#0.3,0.2#
# generate perturbed matrix
nn=60
Jr=np.zeros((nn*2,nn*2))
Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
# Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]-a/nn,Jr[nn:,:nn]+a/nn
# Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
lvec,svr,rvech=la.svd(Jr)
print(' random singular value:',svr[:2])

# add random matrix
x = np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
print('determinant of random matrix:',la.det(x))
x_= x+Jr
lvec_,sv_,rvech_=la.svd(x_)
theo_sv=1/svr[:2]+svr[:2]

## calculate whether there is an additive relationship
overlap_num=1.0-1/svr[0]**2
alpha = np.sqrt(overlap_num)
print('overlap:',alpha**2,1.0-1/svr[0]**2)
print('sigma:',rholimit,'; numsigmas:',sv_[0])

rvec_,rvec=rvech_.T,rvech.T
lvar,rvar=np.zeros_like(lvec_[:,0]),np.zeros_like(rvec_[:,0])

lvar[:nn]=lvec_[:nn,0]-alpha*lvec[:nn,0]
lvar[nn:]=lvec_[nn:,0]-alpha*lvec[nn:,0]
rvar[:nn]=rvec_[:nn,0]-alpha*rvec[:nn,0]
rvar[nn:]=rvec_[nn:,0]-alpha*rvec[nn:,0]

subtilde=x_-sv_[0]*np.reshape(lvec[:,0],(nn*2,1))@np.reshape(rvech[0,:],(1,nn*2))
determinantsubt=la.det(subtilde)
print('determinant of matrix, substract tilde:',determinantsubt)
rhop=1/svr[0]+svr[0]
lvecrec,rvecrec=alpha*lvec[:,0],alpha*rvec[:,0]
suborg=x_-rhop*np.reshape(lvecrec,(2*nn,1))@np.reshape(rvecrec,(1,nn*2))
determinantsubo=la.det(suborg)
print('determinant of matrix, substract original:',determinantsubo)
stlvec,stsv,strvech=la.svd(subtilde)
solvec,sosv,sorvech=la.svd(suborg)
plt.figure()
plt.plot(stsv,'r')
plt.plot(sosv,'b')

# plt.figure()
# n, bins, patches = plt.hist(lvar[:], 100, density=True, facecolor='g', alpha=0.75)
# plt.figure()
# n, bins, patches = plt.hist(rvar[:], 100, density=True, facecolor='g', alpha=0.75)
# plt.figure()
# n, bins, patches = plt.hist(rvec_[:,0], 100, density=True, facecolor='g', alpha=0.75)


 random singular value: [1.01980390e+00 6.07974586e-15]
determinant of random matrix: -1.568320203970407e-28
overlap: 0.038461538461539546 0.038461538461539546
sigma: 3.1819805153394594 ; numsigmas: 2.0388563034338736
determinant of matrix, substract tilde: -8.100835740798812e-28
determinant of matrix, substract original: 4.4758115933812945e-28


Contents
* Can the perturbed matrix $\widetilde{X}$ be represented by basis vectors of low-rank matrix?
* Will the difference between different representations change with the properties of low-rank structure/random matrix (infimum/supramum)
* Can we use either of the (M,N)/(m,n) to represent the perturbed matrix?
* Now we use E/I structure (known already), how about change to use Gaussian distribution?

Edit by Yuxiu

30/10/2020 the $1^{st}$ version V0


In [3]:
# limit of the singular value of the perturbed matrix
# find the largest singular value which are larger than the critical value.
thetac = 2.0
rlarger=svr[np.where(svr>thetac)]
lenc = len(rlarger)
rhop=np.zeros(lenc)
x_tilde_rec=np.zeros_like(x_)
# reconstruct perturbed matrix using basis vectors of low-rank structure
for i in range(lenc):
    rhop[i]=1/svr[i]+svr[i]
    vecoverlap=1-1/svr[i]**2
    lvecrec=np.sqrt(vecoverlap)*lvec[:,i]
    rvecrec=np.sqrt(vecoverlap)*rvec[:,i]
    x_tilde_rec +=rhop[i]*np.reshape(lvecrec,(2*nn,1))@np.reshape(rvecrec,(1,nn*2))#rhop[i]*(lvecrec@rvecrec.T)
print(lenc)


errmat=x_-x_tilde_rec#(rhop[0]*np.reshape(lvecrec,(nn*2,1))@np.reshape(rvecrec,(1,nn*2)))#(sv_[0]*np.reshape(lvec_[:,0],(nn*2,1))@np.reshape(rvec_[:,0].T,(1,nn*2)))
# check the singular value of the Error-matrix
lvecrec,svrec,rvechrec=la.svd(x_tilde_rec)
lvecerr,sverr,rvecherr=la.svd(errmat)
print('recon svrho:',svrec[:5],'; num svrho:',svr[:5])
print('num vec overlap:',np.sum(rvec_[:,0]*rvec[:,0])**2,' theo overlap:',vecoverlap)

print('singular values (largest) of the error matrix:',sverr[:5],sv_[:5])
fig=plt.figure()
ax1=fig.add_subplot(121)
n, bins, patches = ax.hist(sverr, 50, density=True, facecolor='g', alpha=0.75)
ax2=fig.add_subplot(122)
n, bins, patches = ax.hist(sverr, 50, density=True, facecolor='g', alpha=0.75)


1
recon svrho: [6.80859064e+00 6.63107729e-14 6.31349745e-14 5.42914578e-14
 5.31285661e-14] ; num svrho: [6.81175455e+00 5.88095309e-14 5.73088770e-14 4.43769051e-14
 4.24557572e-14]
num vec overlap: 0.9791361252922339  theo overlap: 0.9784482758620688
singular values (largest) of the error matrix: [1.98660329 1.97867277 1.97106312 1.9663761  1.95873549] [6.9618902  1.98562974 1.97727465 1.96883838 1.96626655]


In [4]:
# check whether the Generalized SVD can identically represent the perturbed matrix (all the properties)
# we start from rank-1 perturbation 
m,n=np.ones((nn*2,1)),np.zeros((nn*2,1))
n[:nn,0],n[nn:,0]=JE,-JI
newrep=(1-1/svr[0]**4)*m@n.T/nn
diff_rep=newrep-x_tilde_rec
# plt.imshow(diff_rep)
newrandMat=x_-newrep
lvecnr,svnr,rvechnr=la.svd(newrandMat)
fig=plt.figure()
ax=fig.add_subplot(121)
n, bins, patches = ax.hist(svnr, 50, density=True, facecolor='g', alpha=0.75)

lvecr,svr,rvechr=la.svd(x)
ax1=fig.add_subplot(122)
n, bins, patches = ax1.hist(svr, 50, density=True, facecolor='g', alpha=0.75)

In [5]:
# some simple analysis, how rho_1 changes with JE and JI, how the overlap of vectors changes with JE and JI


In [170]:
# Generate Gaussian vector of E neuron
JEmI,JEpI=1.4,4.4
nn,a,b=1000,0.725,-0.775
yE,yI= np.random.randn(nn),np.random.randn(nn)


JE,JI=(JEpI+JEmI)/2.0,(JEpI-JEmI)/2.0
print(JE,JI)
M1,N1=np.ones(2*nn),np.ones(2*nn)
N1[:nn],N1[nn:]=JE*(1+yE)/nn,-JI*(1+yE)/nn
M2,N2=M1.copy(),N1.copy()
M2[:nn],M2[nn:]=yE,-yE
N2[:nn],N2[nn:]=a*(1+yE)/nn,b*(1+yE)/nn
print(np.mean(a*yE),np.mean(b*yI))

print('overlap M2N2:',np.sum(M2*N2),'; M1N2:',np.sum(M1*N2))

# Jr=np.zeros((nn*2,nn*2))
# Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
# Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]-a/nn,Jr[nn:,:nn]+a/nn
# Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
# lvec,svr,rvech=la.svd(Jr)


2.9000000000000004 1.5000000000000002
-0.03174308978985363 0.030086665202865397
overlap M2N2: 1.4197871535507272 ; M1N2: -0.047810821393803105


In [171]:
J = np.reshape(M1,(2*nn,1))@np.reshape(N1,(1,2*nn))
J +=np.reshape(M2,(2*nn,1))@np.reshape(N2,(1,2*nn))
print(np.shape(J))

(2000, 2000)


In [6]:
# Generate Gaussian vector of E neuron
JEmI,JEpI=-0.5,2.4
nn,a,b=1000,0.6,-0.6
yE,yI= np.random.randn(nn),np.random.randn(nn)


JE,JI=(JEpI+JEmI)/2.0,(JEpI-JEmI)/2.0
print(JE,JI)
M1,N1=np.ones(2*nn),np.ones(2*nn)
N1[:nn],N1[nn:]=JE/nn,-JI/nn
M2,N2=M1.copy(),N1.copy()
M2[:nn],M2[nn:]=yE,-yE
N2[:nn],N2[nn:]=a*yE/nn,b*yE/nn
print(np.mean(a*yE),np.mean(b*yI))

print('overlap M2N2:',np.sum(M2*N2),'; M1N2:',np.sum(M1*N2))

# Jr=np.zeros((nn*2,nn*2))
# Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
# Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]-a/nn,Jr[nn:,:nn]+a/nn
# Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
# lvec,svr,rvech=la.svd(Jr)


0.95 1.45
-0.010506195039353286 -0.012338006217835937
overlap M2N2: 1.1813493730769946 ; M1N2: 0.0


In [7]:
J = np.reshape(M1,(2*nn,1))@np.reshape(N1,(1,2*nn))
J +=np.reshape(M2,(2*nn,1))@np.reshape(N2,(1,2*nn))
print(np.shape(J))

(2000, 2000)


In [189]:
def tanhpfunc(x):
    return np.tanh(x)+1.0
def dynxact(J,x0,timeparams,type='tanhp'):
    tfinal,dt=timeparams[0],timeparams[1]
    nn=np.shape(J)[0]
    nt = int(tfinal/dt)
    xactdyn=np.zeros((nn,nt))
    # print(xFP)
    xactdyn[:,0]=x0
    eps=1e-5
    if type=='tanhp':
        for it in range(1,nt):
            phisyn=np.reshape(tanhpfunc(xactdyn[:,it-1]),(nn,1))
            deltaxact=dt*(-xactdyn[:,it-1]+np.squeeze(J@phisyn))
            xactdyn[:,it]=xactdyn[:,it-1]+deltaxact
    return xactdyn
def dynkappa(M,N,kappa0,timeparams,type='tanhp',express='EI'):
    tfinal,dt=timeparams[0],timeparams[1]
    nn=np.shape(M)[0]
    nt = int(tfinal/dt)
    kappadyn=np.zeros((2,nt))
    kappadyn[:,0]=kappa0
    xactdyn = np.zeros((nn,nt))
    xactdyn[:,0]=kappadyn[0,0]*M[:,0]+kappadyn[1,0]*M[:,1]
    eps=1e-5
    if type=='tanhp':
        for it in range(1,nt):
            xact = kappadyn[0,it-1]*M[:,0]+kappadyn[1,it-1]*M[:,1]
            phisyn=np.reshape(tanhpfunc(xact),(nn,1))
            deltakappa=dt*(-kappadyn[:,it-1]+np.squeeze(N.T@phisyn))
            kappadyn[:,it]=kappadyn[:,it-1]+np.squeeze(deltakappa)
            xactdyn[:,it]=kappadyn[0,it]*M[:,0]+kappadyn[1,it]*M[:,1]
    elif type=='tanh':
        for it in range(1,nt):
            xact = kappadyn[0,it-1]*M[:,0]+kappadyn[1,it-1]*M[:,1]
            phisyn=np.reshape(np.tanh(xact),(nn,1))
            deltakappa=dt*(-kappadyn[:,it-1]+np.squeeze(N.T@phisyn))
            kappadyn[:,it]=kappadyn[:,it-1]+np.squeeze(deltakappa)
            xactdyn[:,it]=kappadyn[0,it]*M[:,0]+kappadyn[1,it]*M[:,1]

    return (kappadyn,xactdyn)
timeparams=np.array([15,0.1])
x0=np.random.rand(nn*2)
# x0[nn:]+=1.0
xact=dynxact(J,x0,timeparams,type='tanhp')
print('axticity:',xact[:3,-1],xact[nn:nn+3,-1])


axticity: [2.5983029  2.61000424 2.69448849] [2.71352655 2.7018252  2.61734092]


In [129]:
# plt.figure()
tt = np.linspace(0,100,1000)
# for i in range(30):
#     plt.plot(tt,xact[i,:],'r')
#     plt.plot(tt,xact[nn+i,:],'b')
frE,frI = xact[:nn,-1],xact[nn:,-1]
fig=plt.figure()
ax=fig.add_subplot(222)
n, bins, patches = ax.hist(frE, 50, density=True)#, facecolor='g', alpha=0.75)
print('mean xE:',np.mean(frE))
ax1=fig.add_subplot(224)
n, bins, patches = ax1.hist(frI, 50, density=True)#, facecolor='g', alpha=0.75)
print('mean xI:',np.mean(frI))
ax=fig.add_subplot(223)
n, bins, patches = ax2.hist((frE-frI)/2.0, 50, density=True)#, facecolor='g', alpha=0.75)
print('mean xI:',np.mean((frE-frI)/2.0))

mean xE: 0.588418496308239
mean xI: 0.5400729430071793
mean xI: 0.024172776650529776


In [190]:
R=2
Mrank,Nrank=np.zeros((nn*2,R)),np.zeros((nn*2,R))
Mrank[:,0],Mrank[:,1],Nrank[:,0],Nrank[:,1]=M1,M2,N1,N2
kappa0=-0.3*np.random.rand(2)
kappadyn,xactdyn=dynkappa(Mrank,Nrank,kappa0,timeparams,type='tanhp',express='EI')
tt = np.linspace(0,timeparams[0],int(timeparams[0]/timeparams[1]))
# plt.figure()
# for i in range(30):
#     plt.plot(tt,xact[i,:],'r')
#     plt.plot(tt,xact[nn+i,:],'b')
frE,frI = xact[:nn,-1],xact[nn:,-1]
# fig=plt.figure()
# ax1=fig.add_subplot(211)
# ax2=fig.add_subplot(212)
n, bins, patches = ax1.hist(frE, 50, density=True, alpha=0.5)#, facecolor='g', alpha=0.75)
print('mean xE:',np.mean(frE))
n, bins, patches = ax2.hist(frI, 50, density=True, alpha=0.5)#, facecolor='g', alpha=0.75)
print('mean xI:',np.mean(frI))
# ax=fig.add_subplot(223)
# n, bins, patches = ax2.hist((frE-frI)/2.0, 50, density=True)#, facecolor='g', alpha=0.75)
# print('mean xI:',np.mean((frE-frI)/2.0))

mean xE: 2.6602015055078385
mean xI: 2.651627852341671


In [193]:
# ax1.set_title(r'$J_E=2.9$'+str(JE)+r',$J_I=1.5$'+str(JI)+',$a=$'+str(a)+',$b=$'+str(b))
ax1.set_title(r'$J_E=2.9$'+', '+r',$J_I=1.5$'+',$a=$'+str(a)+',$b=$'+str(b))
ax1.set_xlabel(r'$x_E^*$',fontsize=14)
ax2.set_xlabel(r'$x_I^*$',fontsize=14)

Text(0.5, 47.04444444444444, '$x_I^*$')

In [177]:
# tt = np.linspace(0,timeparams[0],int(timeparams[0]/timeparams[1]))
# fig=plt.figure()
# ax3=fig.add_subplot(111)
# ax3.plot(tt,kappadyn[0,:],'r',linewidth=1.5,label=r'$\kappa_1^{EI}$')
# ax3.plot(tt,kappadyn[1,:],'b',linewidth=1.5,label=r'$\kappa_2^{EI}$')
# ax3.set_title(r'$J_E=1.2$'+r',$J_I=$'+str(JI)+',$a=$'+str(a)+',$b=$'+str(b))
# ax3.set_xlabel('time',fontsize=14)
# ax3.set_ylabel(r'$\kappa^{EI}$'+' dynamics',fontsize=14)
# plt.grid()
# plt.legend()
# plt.show()

ax3.plot(tt,kappadyn[0,:],'r--',linewidth=1.5)#,label=r'$\kappa_1^{EI}$')
ax3.plot(tt,kappadyn[1,:],'b--',linewidth=1.5)#,label=r'$\kappa_2^{EI}$')